Notes:
- Need to either find a way to handle warnings, or to change all df.append to df.concat
- Params for both APIs are essentially the same, so they could be moved outside of functions and into set-up
- Both APIs could run in parallel on seperate threads to save time waiting for the call delay

## Set-up

In [37]:
# import libraries
from IPython.display import JSON #REMOVE ONCE NOTEBOOK IS COMPLETE
import requests
import pandas as pd
import os
import time

In [38]:
# set constants:
FOURSQUARE_API_KEY = os.getenv('FOURSQUARE_API_KEY')
YELP_API_KEY = os.getenv('YELP_API_KEY')
CALL_DELAY = 0.5 # pause (in s) after each call

In [39]:
# load data
palmaBikes = pd.read_csv('../data/palmaBikes.csv')

# Foursquare

In [42]:
# define Foursquare specific functions

# format lat and long for Foursquare API
def getLatLong(cityBikes_df):
    ll_list = []
    for station in cityBikes_df.index:
        ll_list.append('{},{}'.format(str(cityBikes_df.iloc[station]['latitude']), str(cityBikes_df.iloc[station]['longitude'])))
    return ll_list


def getPOIs_FSQR(cityBikes_df):
    
    # initialize dataframe
    df = pd.DataFrame(columns = ['station_name', 'station_id', 'API', 'id', 'category', 'distance', 'name', 'rating', 'ratings'])
    
    # format latitude and longitude for Foursquare API
    cityBikes_df['ll'] = (pd.Series(getLatLong(palmaBikes)))
    
    for station in cityBikes_df.index:
        
        # make API call
        params = {
            'll': cityBikes_df.iloc[station]['ll'],
            'radius': '1000',
            'fields': ['categories', 'fsq_id', 'distance', 'name', 'rating', 'stats'],
            'sort': 'DISTANCE',
            'limit': '10'
        }
        
        url = f"https://api.foursquare.com/v3/places/search?ll={cityBikes_df.iloc[station]['ll']}&radius={params['radius']}&fields={params['fields'][0]}%2C{params['fields'][1]}%2C{params['fields'][2]}%2C{params['fields'][3]}%2C{params['fields'][4]}%2C{params['fields'][5]}&sort={params['sort']}&limit={params['limit']}"

        headers = {
            "Accept": "application/json",
            "Authorization": FOURSQUARE_API_KEY
        }

        response = requests.request("GET", url, headers=headers).json()
        time.sleep(CALL_DELAY)
        
        for POI in response['results']:
            
            # collecting category, distance, fsq_id, name, rating, ratings
            try:
                category = POI['categories'][0]['name']
            except:
                None
            distance = POI['distance']
            fsq_id = POI['fsq_id']
            name = POI['name']
            try:
                rating = POI['rating']
            except:
                rating = None
            try:
                ratings = POI['stats']['total_ratings']
            except:
                ratings = None
                
            # tag as FSQR
            API = "FSQR"
            
            # add station information
            station_name = cityBikes_df.iloc[station]['name']
            station_id = cityBikes_df.iloc[station]['id']
            
            # save to dataframe
            df = df.append({'station_name': station_name, 'station_id': station_id, 'API': API, 'id': fsq_id, 'category': category,
                            'distance': distance, 'name': name, 'rating': rating,
                            'ratings':ratings}, ignore_index=True)
        
    return df

In [ ]:
results_FSQR = getPOIs_FSQR(palmaBikes)

In [45]:
results_FSQR.head()

,station_name,station_id,API,id,category,distance,name,rating,ratings
0,F. MANUEL HERREROS,abb1d34b9bf558d40cd02e26ab1dc8b3,FSQR,4bc84d218b7c9c748cd337cf,Farmers' Market,43,Mercat de Pere Garau,8.4,75
1,F. MANUEL HERREROS,abb1d34b9bf558d40cd02e26ab1dc8b3,FSQR,4d331da3329e54811bbcb61d,Café,108,Churreria la Artesana,6.2,8
2,F. MANUEL HERREROS,abb1d34b9bf558d40cd02e26ab1dc8b3,FSQR,4f5e6a0ce4b0028cf960560b,Taco Restaurant,299,Chapultepec,9.0,77
3,F. MANUEL HERREROS,abb1d34b9bf558d40cd02e26ab1dc8b3,FSQR,4f6b5cc2e4b0b8a88349b5b2,Grocery Store / Supermarket,313,Alimentos de China,None,None
4,F. MANUEL HERREROS,abb1d34b9bf558d40cd02e26ab1dc8b3,FSQR,4cfe90b759c7b60c34f29f77,Grocery Store / Supermarket,326,Mercadona,5.6,9


# Yelp

In [47]:
#define yelp specific functions

def getPOIs_YELP(cityBikes_df): 

    # initialize dataframe
    df = pd.DataFrame(columns = ['station_name', 'station_id', 'API', 'yelp_id', 'category', 'distance', 'name', 'rating', 'ratings'])
    
    for station in cityBikes_df.index:
        
        # make API call
        params = {
            'latitude': cityBikes_df.iloc[0]['latitude'],
            'longitude': cityBikes_df.iloc[0]['longitude'],
            'radius': '1000',
            'limit': '10'
        }
        url = (f"https://api.yelp.com/v3/businesses/search?latitude={params['latitude']}&longitude={params['longitude']}&radius={params['radius']}&sort_by=distance&limit={params['limit']}")
        
        headers = {
            "accept": "application/json",
            "Authorization": "bearer " + YELP_API_KEY
        }

        response = requests.get(url, headers=headers).json()
        time.sleep(CALL_DELAY)
                
        for POI in response['businesses']:
            
            # collect id, category, distance, name, rating, ratings
            yelp_id = POI['id']
            try:
                category = POI['categories'][0]['title']
            except:
                None
            distance = POI['distance']
            name = POI['name']
            rating = POI['rating']
            ratings = POI['review_count']
            
            # tag as Yelp
            API = "YELP"
            
            # add station information
            station_name = cityBikes_df.iloc[station]['name']
            station_id = cityBikes_df.iloc[station]['id']
            
            # save to dataframe
            df = df.append({'station_name': station_name, 'station_id': station_id, 'API': API, 'yelp_id': yelp_id, 'category': category, 'distance': distance, 
                            'name': name, 'rating': rating,
                            'ratings':ratings}, ignore_index=True)
    return df

In [ ]:
results_YELP = getPOIs_YELP(palmaBikes)

In [49]:
results_YELP.head()

,station_name,station_id,API,yelp_id,category,distance,name,rating,ratings
0,F. MANUEL HERREROS,abb1d34b9bf558d40cd02e26ab1dc8b3,YELP,BWWXOHCcfSNXl3yeGcErsw,Chinese,324.543739,Shi Shan Xuan,5.0,3
1,F. MANUEL HERREROS,abb1d34b9bf558d40cd02e26ab1dc8b3,YELP,e3F0HeIfhCByxFN9hUP_cA,Bars,406.299598,Las Caribenas,5.0,1
2,F. MANUEL HERREROS,abb1d34b9bf558d40cd02e26ab1dc8b3,YELP,8yYthxSx297ic_pIG5D3Lg,Tapas Bars,406.663624,Dehesa Santa Maria,4.0,1
3,F. MANUEL HERREROS,abb1d34b9bf558d40cd02e26ab1dc8b3,YELP,4tGOSHJlKK2MXkwszcnbdQ,Bars,423.613087,Bar Monaco,4.0,1
4,F. MANUEL HERREROS,abb1d34b9bf558d40cd02e26ab1dc8b3,YELP,JqjGid2vEcHBvxSbL2Oi5w,Spanish,444.985865,Palacio Chino,5.0,1


# Comparing Results

Which API provided you with more complete data? Provide an explanation. 

Get the top 10 restaurants according to their rating